In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Manager

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optimizer
import torchvision
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

from fastprogress import master_bar, progress_bar

import math
from PIL import Image
import random
import numpy as np
from skimage import color

import sys

import os
import cv2
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor
import matplotlib.patches as patches

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from PIL import Image
import requests
from transformers import SamModel, SamProcessor

import albumentations as A
import albumentations.augmentations.functional as F

def solve_utf8_problem():
  # in case that : NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
  import locale
  locale.getpreferredencoding = lambda: "UTF-8"

random_state = 78
torch.seed = random_state

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Water Bodies Dataset/Images')
os.chdir('/content/drive/MyDrive/Water Bodies Dataset/Masks')

root_dataset = '/content/drive/MyDrive/Water Bodies Dataset/'
image_data_path='/content/drive/MyDrive/Water Bodies Dataset/Images'
mask_data_path='/content/drive/MyDrive/Water Bodies Dataset/Masks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)

processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

In [ ]:
def get_bounding_box(ground_truth_map):
  # get bounding box from mask
  y_indices, x_indices = np.where(ground_truth_map > 100)
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = ground_truth_map.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  bbox = [x_min, y_min, x_max, y_max]

  return bbox

In [ ]:
def preprocess_mask(image):
  return image

class SegmentationDataset(Dataset):
    def __init__(self, root_dir, processor, transform=None):
        self.root_dir = root_dir
        self.image_folder = os.path.join(root_dir, "Images")
        self.mask_folder = os.path.join(root_dir, "Masks")
        self.image_list = os.listdir(self.image_folder)
        self.transform = transform
        self.processor = processor

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_list[idx])
        mask_name = os.path.join(self.mask_folder, self.image_list[idx])

        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_name, cv2.IMREAD_UNCHANGED)
        mask = preprocess_mask(mask)

        transformed = A.Resize(1024, 1024)(image=image, mask=mask)
        image = transformed["image"]
        mask = transformed["mask"]

        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]

        # get bounding box prompt
        prompt = get_bounding_box(mask)

        # prepare image and prompt for the model
        inputs = self.processor(image, input_boxes=[[prompt]], return_tensors="pt")

        # remove batch dimension which the processor adds by default
        inputs = {k:v.squeeze(0) for k,v in inputs.items()}

        # add ground truth segmentation
        inputs["ground_truth_mask"] = mask

        return inputs

In [ ]:
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=45, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        # A.Resize(1024, 1024),
        ToTensorV2(),
    ]
)



dataset = SegmentationDataset(root_dataset,processor=processor, transform=train_transform)
train_valid_ratio = 1/10
train_idx, validation_idx = train_test_split(np.arange(len(dataset)), test_size=train_valid_ratio, random_state=random_state)

training_data = torch.utils.data.Subset(dataset, train_idx)
test_data = torch.utils.data.Subset(dataset, validation_idx)

In [ ]:
num_samples_to_visualize = 3

for i in range(num_samples_to_visualize):
    image, mask = training_data[i]

    # Convert the PyTorch tensors back to PIL images for visualization

    # Plot the original image
    plt.subplot(num_samples_to_visualize, 2, 2 * i + 1)
    plt.imshow(image)
    plt.title('Original Image')
    plt.axis('off')

    # Plot the corresponding mask
    plt.subplot(num_samples_to_visualize, 2, 2 * i + 2)
    plt.imshow(mask, cmap='gray')  # Assuming masks are grayscale
    plt.title('Mask')
    plt.axis('off')

plt.show()

ValueError: ignored

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=2)
test_dataloader = DataLoader(test_data, batch_size=2)

ValueError: ignored

In [ ]:
class Dice(nn.Module):
    def __init__(self):
        super(Dice, self).__init__()

    def forward(self, predictions, targets, smooth=1):
        intersection = (predictions * targets).sum()
        union = predictions.sum() + targets.sum() + smooth

        dice_coefficient = (2.0 * intersection + smooth) / union
        dice_loss = 1 - dice_coefficient

        return dice_coefficient

class IoU(nn.Module):
    def __init__(self):
        super(IoU, self).__init__()

    def forward(self, predictions, targets, smooth=1):
        intersection = (predictions * targets).sum()
        union = predictions.sum() + targets.sum() - intersection + smooth

        iou = (intersection + smooth) / union
        iou_loss = 1 - iou

        return iou
# Example Usage:
# Assuming you have a pre-trained model and a DataLoader for your dataset
# model = your_pretrained_model
# dataloader = your_dataloader

# Define your loss functions
dice_fn = Dice()
iou_fn = IoU()

In [ ]:
for parameter in model.prompt_encoder.parameters():
  parameter.require_grad = False

for parameter in model.vision_encoder.parameters():
  parameter.require_grad = False

for parameter in model.mask_decoder.parameters():
  parameter.require_grad = True

In [ ]:
pip install monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00


In [ ]:
from torch.optim import Adam
import monai

# Define optimizer
optimizer = Adam(model.mask_decoder.parameters(), lr=1e-5, weight_decay=0)
seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from statistics import mean
import torch
from torch.nn.functional import threshold, normalize


bce = torch.nn.BCELoss()
# Training loop
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    epoch_losses = []
    for batch in tqdm(train_dataloader):
      # forward pass
      outputs = model(pixel_values=batch["pixel_values"].to(device),
                      input_boxes=batch["input_boxes"].to(device),
                      multimask_output=False)

      # compute loss
      predicted_masks = outputs.pred_masks.squeeze(1)
      ground_truth_masks = batch["ground_truth_mask"].float().to(device)
      loss = seg_loss(predicted_masks, ground_truth_masks.unsqueeze(1))

      # backward pass (compute gradients of parameters w.r.t. loss)
      optimizer.zero_grad()
      loss.backward()

      # optimize
      optimizer.step()
      epoch_losses.append(loss.item())

    print(f'EPOCH: {epoch}')
    print(f'Mean loss: {mean(epoch_losses)}')
    print(dice)
    print(iou)





  0%|          | 0/1283 [00:00<?, ?it/s]


ValueError: ignored